In [1]:
import warnings
import pprint
import skrebate
import imblearn
import random
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, LeaveOneGroupOut, cross_validate, cross_val_predict
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import QuantileTransformer, quantile_transform, RobustScaler
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')



/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
combined_master = pd.read_csv('./mIHC/test1_newLabels/test1&3_master_ready.tsv.gz', '\t', index_col=0)
combined_master

,test_no,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
0,test1,4381R6_53753,4381R6,F,0.111675,0.171717,0.108491,0.273196,0.133333,0.136150,...,0.001208,0.009804,0.015385,0.010050,1385.771930,3962.368421,0.000000,171,0.333333,0.151042
1,test3,SMA37F_32367,SMA37F,G,0.013072,0.000000,0.000000,0.000000,0.000000,0.005682,...,0.000000,0.000000,0.000000,0.000000,1070.000000,4240.500000,0.000000,8,0.000000,0.000000
2,test1,4411R4_18331,4411R4,B,0.015504,1.369127,1.126506,0.344538,0.025641,0.945946,...,0.065574,0.000000,0.000000,0.118421,2172.116541,1882.721805,0.000000,266,0.201835,0.006024
3,test3,SMA54F_1539,SMA54F,G,0.086957,0.111111,0.368932,0.173077,0.004505,0.000000,...,0.000000,0.014085,0.014493,0.000000,984.500000,223.500000,0.009709,12,0.150000,0.000000
4,test1,4413R1_2105,4413R1,F,0.202532,0.180723,0.246753,0.234043,0.203704,0.032680,...,0.000000,0.000000,0.012658,0.000000,188.007194,479.100719,0.550000,139,0.392157,0.368098
5,test1,4380R6_13845,4380R6,F,0.710059,0.645714,0.584158,0.851190,0.320513,0.931937,...,0.029268,0.036667,0.005952,0.015228,1156.379121,1775.186813,1.335135,182,1.302158,0.745562
6,test1,4381R6_46009,4381R6,D,0.000000,0.000000,5.528302,0.231959,0.010256,4.004695,...,0.000000,0.000000,0.000000,0.000000,2527.220472,3408.422572,0.000000,381,0.020202,0.000000
7,test1,4547R3_1044,4547R3,F,0.000000,0.035714,0.034091,0.035211,0.010526,0.004525,...,0.000000,0.000000,0.000000,0.000000,3141.260563,220.943662,0.068376,142,0.015152,0.000000
8,test3,SMA40F_19470,SMA40F,A,0.145833,1.496774,2.045161,0.007353,0.000000,0.093897,...,0.062500,0.034884,0.044444,0.000000,939.629870,2876.051948,0.744186,154,0.438356,0.000000
9,test3,SMA60F_720,SMA60F,E,0.152074,0.643243,12.910526,1.536585,0.254464,0.410891,...,0.012658,0.123457,0.013245,0.025974,537.804167,129.704167,0.413534,240,2.277778,3.552000


In [4]:
combined_master['class'].value_counts()

F    10150
E     9255
G     9058
B     7843
H     7118
A     6434
D     5275
C     3150
Name: class, dtype: int64

In [5]:
predicted = pd.read_csv('./Galaxy10-[Model_Validation_on_data_9_and_data_6_-_By_patient].tabular', sep='\t')
predicted

,Predicted
0,F
1,G
2,B
3,G
4,F
5,F
6,D
7,F
8,A
9,E


In [6]:
result_combo = combined_master.iloc[:, :4]
result_combo['Predicted'] = predicted['Predicted']
result_combo

,test_no,obj_ID,patient_ID,class,Predicted
0,test1,4381R6_53753,4381R6,F,F
1,test3,SMA37F_32367,SMA37F,G,G
2,test1,4411R4_18331,4411R4,B,B
3,test3,SMA54F_1539,SMA54F,G,G
4,test1,4413R1_2105,4413R1,F,F
5,test1,4380R6_13845,4380R6,F,F
6,test1,4381R6_46009,4381R6,D,D
7,test1,4547R3_1044,4547R3,F,F
8,test3,SMA40F_19470,SMA40F,A,A
9,test3,SMA60F_720,SMA60F,E,E


## How the model performs on each class overall?

In [7]:
# balanced accuracy overall
b_accuracy = balanced_accuracy_score(result_combo['class'], result_combo['Predicted'])
b_accuracy

0.8962043127619868

In [8]:
# confusion matrix overall
mtrix_all = confusion_matrix(result_combo['class'], result_combo['Predicted'],
                            labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
pd.DataFrame(mtrix_all, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,5938,161,24,77,193,11,15,15
1,165,6993,49,158,384,55,10,29
2,105,88,2825,17,107,5,2,1
3,90,163,14,4737,229,22,17,3
4,350,516,46,274,7722,251,59,37
5,19,35,12,56,301,9166,306,255
6,15,25,3,22,71,402,8376,144
7,41,9,5,23,65,524,58,6393


In [9]:
# precision for each class
precision = np.diag(mtrix_all) / mtrix_all.sum(axis=1).astype(float)
pd.DataFrame([precision], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,0.92291,0.891623,0.896825,0.898009,0.83436,0.903054,0.924707,0.898146


In [10]:
# recall for each class
recall = np.diag(mtrix_all) / mtrix_all.sum(axis=0).astype(float)
pd.DataFrame([recall], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

,A,B,C,D,E,F,G,H
0,0.883237,0.875219,0.948623,0.88311,0.85119,0.878306,0.94719,0.92962


## How the model performs on each patient?

In [11]:
accuracies = []
for p, p_group in result_combo.groupby(['patient_ID']):
    b_accuracy = balanced_accuracy_score(p_group['class'], p_group['Predicted'])
    accuracies.append(b_accuracy)
    mtrix = confusion_matrix(p_group['class'], p_group['Predicted'], 
                             labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    precisions = np.diag(mtrix) / mtrix.sum(axis=1).astype(float)
    recalls = np.diag(mtrix) / mtrix.sum(axis=0).astype(float)
    print("For patient %s:"%p)
    print("\tBalanced_accuracy:%f" %b_accuracy)
    print("\tPrecisions for each class:" )
    print(pd.DataFrame([precisions], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\tRecalls for each class:" )
    print(pd.DataFrame([recalls], columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\tConfusion matrix:" )
    print(pd.DataFrame(mtrix, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))
    print("\n\n")
print("Mean balanced accuracy: %f" %(sum(accuracies)/float(len(accuracies))))    

For patient 4350R3:
	Balanced_accuracy:0.922141
	Precisions for each class:
          A         B    C         D         E         F         G         H
0  0.742857  0.984496  1.0  0.711111  0.971223  0.977143  0.992038  0.998261

	Recalls for each class:
          A         B         C    D         E         F        G         H
0  0.945455  0.846667  0.833333  1.0  0.882353  0.997085  0.99362  0.986254

	Confusion matrix:
    A    B  C   D    E    F    G    H
0  52   17  0   0    1    0    0    0
1   0  127  0   0    1    0    1    0
2   0    0  5   0    0    0    0    0
3   0    0  0  32   13    0    0    0
4   0    3  1   0  135    0    0    0
5   2    3  0   0    3  684    2    6
6   1    0  0   0    0    2  623    2
7   0    0  0   0    0    0    1  574



For patient 4380R6:
	Balanced_accuracy:0.917342
	Precisions for each class:
          A         B         C         D         E         F         G  \
0  0.958571  0.895714  0.980392  0.996914  0.855714  0.841429  0.818571   



0  0.879668  

	Confusion matrix:
     A    B   C    D    E    F    G    H
0  954    2  20   42  161    7    4   14
1   12  702  21   90  330   44    4   28
2    0    0  65    0    4    1    1    0
3    0    0   0  260    9   10    0    2
4    0    0   0   17  775   83    3   22
5    1    0   0    0    2  611   15   21
6    0    0   0    2    1    0  647    0
7    0    0   0    0    1    6    7  636



For patient SMA54F:
	Balanced_accuracy:0.896289
	Precisions for each class:
          A         B     C         D         E         F         G         H
0  0.996764  0.888545  0.75  0.820896  0.984877  0.947692  0.996923  0.784615

	Recalls for each class:
          A         B         C         D         E         F         G  \
0  0.919403  0.989655  0.666667  0.932203  0.928699  0.831309  0.961424   

         H  
0  0.98646  

	Confusion matrix:
     A    B  C   D    E    F    G    H
0  308    0  0   1    0    0    0    0
1   24  287  1   3    6    2    0    0
2    0    0  6   0    

## Learning curve

In [3]:
curve = pd.read_csv('./Galaxy12-[Model_Validation_on_data_9_and_data_2_--Learning_curve_by_patient].tabular', sep='\t')
curve

,train_sizes_abs,mean_train_scores,std_train_scores,mean_test_scores,std_test_scores
0,5599,0.986433,0.001680,0.877656,0.067400
1,11198,0.972978,0.002096,0.876594,0.069761
2,16797,0.966614,0.002678,0.879799,0.069262
3,22396,0.962851,0.002925,0.880995,0.069056
4,27996,0.960747,0.002797,0.880987,0.068048
5,33595,0.959170,0.002918,0.881325,0.066225
6,39194,0.957882,0.003097,0.882824,0.066677
7,44793,0.957114,0.003194,0.880210,0.067552
8,50392,0.956375,0.003335,0.879759,0.067202
9,55992,0.955984,0.003207,0.880727,0.068371


In [4]:

data1 = go.Scatter(
        x = curve['train_sizes_abs'],
        y = curve['mean_train_scores'],
        mode = "lines",
        name = "Train scores",
)
data2 = go.Scatter(
    x = curve['train_sizes_abs'],
    y = curve['mean_test_scores'],
    mode = "lines",
    name="Test scores"
)
layout = dict(
    xaxis=dict(
        title='No. of samples'
    ),
    yaxis=dict(
        title='Balanced Accuracy'
    )
)
fig = go.Figure(data=[data1, data2], layout=layout)
iplot(fig)